# Transformer based models

### Importing the required libraries

In [7]:
# ! pip install allennlp
# ! pip install allennlp-models
# ! pip install nltk==3.5

import io
import numpy as np
# from google_trans_new import google_translator 
!pip install mtranslate
from scipy.spatial.distance import cosine
import json 
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk import word_tokenize
from itertools import groupby 
from spacy.lang.hi import STOP_WORDS as STOP_WORDS_HI

from tqdm import tqdm
import pandas as pd

!pip install indic-nlp-library
import indicnlp
from indicnlp.tokenize import indic_tokenize 
!pip install -U torch
import torch
!pip install -U transformers
!pip install -U sentence-transformers
import transformers
from sentence_transformers import SentenceTransformer,util
import numpy as np
# !pip install google_trans_new
# from google_trans_new import google_translator
!pip install stanfordnlp
import stanfordnlp 
stanfordnlp.download('hi')
nlp = stanfordnlp.Pipeline(processors = "tokenize,pos",lang = 'hi')

  Created wheel for mtranslate: filename=mtranslate-1.8-cp37-none-any.whl size=3697 sha256=79629ad536f21f0bb6e4feb44bf2618a98a00db2843e7bf76dfd7641c2b19e03
  Stored in directory: /root/.cache/pip/wheels/eb/fb/4a/f63e74cbfb835161f3f2d1a6b607b137d344a5cb6d8c1303fa
Successfully built mtranslate
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 40kB 6.8MB/s 
     |████████████████████████████████| 9.2MB 26.4MB/s 
     |████████████████████████████████| 552kB 49.8MB/s 
  Created wheel for sphinx-argparse: filename=sphinx_argparse-0.2.5-cp37-none-any.whl size=11552 sha256=3437fac396ddc89e2c511f2b1e183139ea8eba2f40425c885dd80178b87f95ac
  Stored in directory: /root/.cache/pip/wheels/2a/18/1b/4990a1859da4edc77ab312bc2986c08d2733fb5713d06e44f5
Successfully built sphinx-argparse
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found ex

100%|██████████| 208M/208M [00:35<00:00, 5.91MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/hi_hdtb_models.zip
Extracting models file for: hi_hdtb
Cleaning up...Done.
Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_tokenizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/hi_hdtb_models/hi_hdtb.pretrain.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
Done loading processors!
---


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from allennlp.predictors import Predictor
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-model-2020.02.10.tar.gz")

In [9]:
# model = SentenceTransformer('stsb-xlm-r-multilingual')
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)
model.to(device)
# model = SentenceTransformer('msmarco-distilbert-base-v2')
# model = SentenceTransformer('LaBSE')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

# Loading Corpus data (universal)

In [2]:
import json
# Making the test_dict
actors_sent, actors_trip = [],[]
with open('/content/drive/MyDrive/train_data/actors.jsonl', 'r', encoding='utf8') as read_file:
    for line in read_file:
        data = json.loads(line)

        
        tuple_trip_list = []
        if len(data['triples'])>20:
          continue
        for triple in data['triples']:
            tuple_trip_list.append(tuple(triple))
        actors_trip.append(tuple_trip_list)
        actors_sent.append([data["sentence"]])
        #processing on json data dictionary
        pass
len(actors_sent), len(actors_trip)

(3093, 3093)

In [3]:
len(actors_sent), len(actors_trip)

(5907, 5907)

In [17]:
actors_trip[0]

[('Ashish Kakkad', 'date of birth', '1971'),
 ('Ashish Kakkad', 'date of death', '02 November 2020'),
 ('Ashish Kakkad', 'instance of', 'human'),
 ('Ashish Kakkad', 'sex or gender', 'male'),
 ('Ashish Kakkad', 'occupation', 'screenwriter'),
 ('Ashish Kakkad', 'occupation', 'film producer'),
 ('Ashish Kakkad', 'occupation', 'film director'),
 ('Ashish Kakkad', 'occupation', 'actor'),
 ('Ashish Kakkad', 'occupation', 'artist'),
 ('Ashish Kakkad', 'country of citizenship', 'India'),
 ('Ashish Kakkad', 'place of death', 'Kolkata'),
 ('Ashish Kakkad', 'given name', 'Ashish')]

In [5]:
actors_sent[0]

['आशीष कक्कड़ ( 21 मई 1971 - 2 नवंबर 2020 ) एक भारतीय फिल्म निर्देशक , लेखक , अभिनेता और आवाज कलाकार थे ।']

# Alignment (keyphrase)

Finding Top K matches

In [10]:
from nltk.util import ngrams
from nltk import RegexpParser

def extract_pos(doc):
    pos = [(wrd.text,wrd.pos) for sent in doc.sentences for wrd in sent.words]
    return pos

def preprocess(triple_list):
  new_triple_list = []
  for s,p,o in triple_list:
    if p == 'date of birth' or p == 'date of death' or 'name' in p or p == 'sex or gender' or o == 'British India' or o == 'Dominion of India':
      continue
    else:
      new_triple_list.append((s,p,o))
  return new_triple_list

def matches_kp(sentences,triples):       # Matches based on keyphrase
    matches_dict = {}                 # Dictionary to store the sentences with the matching triples. key = sent, value = triple
    for sent in sentences:
      sentence2 =  sent
      hindi = nlp(sentence2)
      extract_pos(hindi)
      pos_tags = extract_pos(hindi)
      patterns = "NP: {<JJ>*<NN|NNS|NNC|NNCS|NNP|NNPS|NNPC|NNPCS>+}"
      # patterns = "NP:{(<JJ>* <NN.*>+ <PSP>)? <JJ>* <NN.*>+}"
      PChunker = RegexpParser(patterns)
      pos_tags = extract_pos(hindi)
      output = PChunker.parse(pos_tags)
      keyphrases = []
      for subtree in output.subtrees(filter=lambda t: t.label() == 'NP'):
          keyphrases.append((' '.join([x[0] for x in subtree])))
      triple_list = triples
      triple_list = preprocess(triple_list)
      matches = []
      # keyphrases = ['ईरानी ऑस्ट्रियाई राजनीतिक कार्यकर्ता है']
      for kp in keyphrases:
        words = [w for w in indic_tokenize.trivial_tokenize(kp, lang = 'hi') if w not in STOP_WORDS_HI]
        k = len(words)
        if k==0:
          continue
        bigrams = ngrams(words,2)
        trigrams = ngrams(words,3)
        if len(words) == 1:
          bigrams,trigrams = [(words)],[(words)]
        matches_for_a_keyphrase = []
        for tup in bigrams:
          
          sbi = ' '.join(list(tup))
          for s,p,o, in triple_list:
            embedding1 = model.encode(sbi, convert_to_tensor=True)
            embedding2 = model.encode(p+" "+o, convert_to_tensor=True)
            cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
            if cosine_scores.item() > 0.4:
              matches_for_a_keyphrase.append([cosine_scores.item(),(s,p,o),kp])
        for tup in trigrams:
          sbi = ' '.join(list(tup))
          for s,p,o in triple_list:
            embedding1 = model.encode(sbi, convert_to_tensor=True)
            embedding2 = model.encode(p+" "+o, convert_to_tensor=True)
            cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
            if cosine_scores.item() > 0.4:
              matches_for_a_keyphrase.append([cosine_scores.item(),(s,p,o),kp])
      
        
        matches_for_a_keyphrase.sort(reverse = True)
        triple_matches = []
        # Now, we have to keep only (s,p,o) from matches_for_a_keyphrase in triple_matches
        for _,triple,_ in matches_for_a_keyphrase:
          triple_matches.append(triple) 
        matches.extend(triple_matches[:min(k,2)])
        
      matches.sort(reverse = True)
      matches_dict[sent] = list(set(matches))   
    return matches_dict

# Execute

In [4]:
len(actors_trip[0])

12

In [11]:
from tqdm import tqdm
matches_actors_corpus = {}
for sent_list, triple_list in tqdm(zip([actors_sent[0]], [actors_trip[0]])):
    for k, v in matches_kp(sent_list, triple_list).items():
        matches_actors_corpus[k] = v

1it [00:03,  3.03s/it]


In [ ]:
# matches_actors_corpus = {}
# for sent_list, triple_list in tqdm(zip([actors_sent[7]], [actors_trip[7]])):
#     for k, v in matches_kp(sent_list, triple_list).items():
#         matches_actors_corpus[k] = v

In [12]:
matches_actors_corpus

{'आशीष कक्कड़ ( 21 मई 1971 - 2 नवंबर 2020 ) एक भारतीय फिल्म निर्देशक , लेखक , अभिनेता और आवाज कलाकार थे ।': [('Ashish Kakkad',
   'occupation',
   'actor'),
  ('Ashish Kakkad', 'occupation', 'screenwriter'),
  ('Ashish Kakkad', 'occupation', 'artist'),
  ('Ashish Kakkad', 'country of citizenship', 'India'),
  ('Ashish Kakkad', 'occupation', 'film director')]}

In [ ]:
## post processing
for sent in matches_keyphrases:
  triples = matches_keyphrases[sent]
  new_trips = []
  for triple in triples:
    if triple[1] == 'date of birth' or triple[1] == 'date of death' or 'name' in triple[1] or triple[1] == 'sex or gender' or triple[2] == 'British India' or triple[2] == 'Dominion of India':
      pass
    else:
      new_trips.append(triple)
  matches_keyphrases[sent] = new_trips


### Output

In [ ]:
len(matches_act),len(matches_cric),len(matches_pol)

So, 7 sentences didn't find any matches at all.Let's see which ones

In [ ]:
for sent in politicians_test_dict:
  if sent not in matches_pol:
    print(sent," : ", politicians_test_dict[sent] )

Saving as json

In [ ]:
# for sent in actors_test_dict:
#   actors_test_dict[sent] = list(actors_test_dict[sent])
# for sent in cricketers_test_dict:
#   cricketers_test_dict[sent] = list(cricketers_test_dict[sent])
# for sent in politicians_test_dict:
#   politicians_test_dict[sent] = list(politicians_test_dict[sent])

# for sent in matches_act:
#   matches_act[sent] = list(matches_act[sent])
# for sent in matches_cric:
#   matches_cric[sent] = list(matches_cric[sent])
for sent in matches_pol:
  matches_pol[sent] = list(matches_pol[sent])

In [ ]:
test_dict

In [ ]:
# # saving the input test files
# with open("/content//drive//MyDrive//Transformer Output//actors_test.json", "w") as outfile: 
#     json.dump(actors_test_dict,outfile)
# with open("/content/drive/MyDrive/Transformer Output/cricketers_test.json", "w") as outfile: 
#     json.dump(cricketers_test_dict, outfile)
# with open("/content/drive/MyDrive/Transformer Output/politicians_test.json", "w") as outfile: 
#     json.dump(politicians_test_dict, outfile)
with open("/content/drive/MyDrive/Transformer Output/test_dict.json", "w") as outfile: 
    json.dump(test_dict, outfile)

# #saving the output files
# with open("/content/drive/MyDrive/Transformer Output/actors_matches.json", "w") as outfile: 
#     json.dump(matches_act, outfile)
# with open("/content/drive/MyDrive/Transformer Output/cricketers_matches.json", "w") as outfile: 
#     json.dump(matches_cric, outfile)
# with open("/content/drive/MyDrive/Transformer Output/matches_final_NER.json", "w") as outfile: 
#     json.dump(matches_final_NER, outfile)
# with open("/content/drive/MyDrive/Transformer Output/matches_keyphrases.json", "w") as outfile: 
#     json.dump(matches_keyphrases, outfile)

TypeError: ignored

#### We do find quite a few of the matching triples to be relevant to the sentence. But, there are a few irrelevant matches as well.
Upon analysis, we think the word overlap is working better than the vector similarity approach. A possible reason can be that when we simply average out the words in a sentence, and when we average out the words in the triples and then find the similarity between these two averages, some semantic information is lost. So, triples that should have been irrelevant are also found as similar. As the word overlap method is a strictly string overlap, the relevance is much stronger.